# Test Mesh Simplification

In [1]:
import numpy as np
import os
import plotly.graph_objects as go

import planeslam.io as io
import planeslam.general as general
from planeslam.mesh import LidarMesh
from planeslam.clustering import cluster_mesh_graph_search, mesh_cluster_pts, find_cluster_boundary

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)

In [3]:
P = PCs[0]

# Convert points to ENU
P = general.NED_to_ENU(P)

In [4]:
# Downsample the points
P = general.downsample(P, factor=2, axis=0)

In [15]:
# Plot the scan
fig = go.Figure(data=general.pc_plot_trace(P))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [5]:
mesh = LidarMesh(P)
mesh.prune(edge_len_lim=10)

In [6]:
fig = go.Figure(data=mesh.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
def cluster_mesh_graph_search(mesh, normal_match_thresh=0.866, min_cluster_size=20):
    # Compute surface normals
    normals = mesh.compute_normals()

    # Graph search
    clusters = []  # Clusters are idxs of triangles, triangles are idxs of points
    # For each cluster, also maintain a polygon representation
    cluster_polys = []
    to_cluster = set(range(len(mesh.DT.simplices)))

    while to_cluster:
        root = to_cluster.pop()
        cluster_normal = normals[root,:]

        cluster = [root]
        poly = 
        search_queue = set(mesh.tri_nbr_dict[root])
        search_queue = set([x for x in search_queue if x in to_cluster])  # Don't search nodes that have already been clustered

        while search_queue:
            i = search_queue.pop()
            if np.dot(normals[i,:], cluster_normal) > normal_match_thresh:
                # Add node to cluster and remove from to_cluster
                cluster.append(i)
                to_cluster.remove(i)
                # Add its neighbors (that are not already clustered) to the search queue
                search_nbrs = mesh.tri_nbr_dict[i].copy()
                search_nbrs = [x for x in search_nbrs if x in to_cluster]
                search_queue.update(search_nbrs)

        if len(cluster) >= min_cluster_size:
            clusters.append(cluster)

    avg_normals = len(clusters) * [None]
    for i, c in enumerate(clusters):
        avg_normals[i] = normalize(np.mean(normals[c], axis=0))

    return clusters, avg_normals

In [7]:
# Cluster the mesh and identify boundary/non-boundary points of clusters
clusters, avg_normals = cluster_mesh_graph_search(mesh)

In [8]:
cluster_pts = mesh_cluster_pts(mesh, clusters[0])

In [9]:
fig = go.Figure(data=general.pc_plot_trace(cluster_pts))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()

In [12]:
bd_idxs = find_cluster_boundary(clusters[0], mesh)
bd_pts = mesh.P[bd_idxs,:]

In [13]:
fig = go.Figure(data=general.pc_plot_trace(bd_pts))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)))
fig.show()